# Install all necessary packages
- **pandas:** for data manipulation and analysis, particularly with DataFrames.
- **numpy:** for large, multi-dimensional arrays and matrices
- **seaborn:** A statistical data visualization library based on Matplotlib
- **matplotlib:** or creating static, interactive, and animated visualizations.
- **prettytable:** for creating ASCII tables to better display data.
- **missingno:** for visualizing missing values in datasets.

In [ ]:
!pip3 install pandas numpy seaborn matplotlib prettytable missingno mpl-tools

# Import packages

In [46]:
import random
import numpy as np 
import pandas as pd
from pandas.plotting import parallel_coordinates

from prettytable import PrettyTable

import seaborn as sns
import missingno as msno
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# 0.) Prepare dataset

In [25]:
# CSV file name
file_name = "Most Streamed Spotify Songs 2024.csv"

# Set display options for pandas
pd.set_option('display.max_columns', None)

# Load CSV file into DataFrame
df = pd.read_csv('Most Streamed Spotify Songs 2024.csv', encoding="ISO-8859-1")


# 1.) Data analysis

## 1.1) Dataset

In [ ]:
df.head() # Prints the first 5 row and column in the data

In [ ]:
df.describe() # Provides a summary of statistics for numerical columns in the DataFrame.

In [ ]:
df.info() # Provides a small summary about the data

## 1.2) Plot missing values

In [ ]:
# Identify number of missing values in dataset
df_missing = pd.DataFrame(df.isnull().sum()).reset_index().rename(columns={'index': 'Track', 0: 'Missing Values'})
df_missing.drop(index=df.index[0], axis=0, inplace=True)
df_missing[df_missing['Missing Values']>0].sort_values(by='Missing Values', ascending=False)

In [ ]:
msno.bar(df)

plt.show()

In [ ]:
# Identify duplicate values
df_duplicates = pd.DataFrame(df.duplicated()).rename(columns={0: 'duplicate'})
df_duplicates[df_duplicates['duplicate'] == True]

## 1.3) Data types of each feature

In [ ]:
table = PrettyTable()
table.field_names = ['Feature', 'Data Type']

for column in df.columns:
    column_dtype = str(df[column].dtype)
    table.add_row([column, column_dtype])

print(table)

# 2.) Feature Engineering and data cleanup

In [ ]:
def feature_engineering(df):
    # Convert 'release date' to datetime format
    df['Release Date'] = pd.to_datetime(df['Release Date'], format='%m/%d/%Y')
    
    # Create a new column 'Year' by extracting the year from 'release date'
    df['Year'] = df['Release Date'].dt.year
    
    # Remove dataset with all empty value
    df = df.drop(columns=["TIDAL Popularity"])
    
    # All column that need data conversion to integer
    integer_columns_to_convert = ['Spotify Streams', 'Spotify Playlist Count', 'Spotify Playlist Reach', 'YouTube Views', 'YouTube Likes', 'TikTok Posts', 'TikTok Likes', 'TikTok Views', 'YouTube Playlist Reach', 'AirPlay Spins', 'Deezer Playlist Reach', 'Pandora Streams', 'Pandora Track Stations', 'Soundcloud Streams', 'Shazam Counts']
    
    # All column that need data conversion to float
    float_columns_to_convert = ['Track Score', 'Spotify Popularity', 'Apple Music Playlist Count', 'SiriusXM Spins', 'Amazon Playlist Count']
    
    # Replace null values with 0
    df[integer_columns_to_convert + float_columns_to_convert] = df[integer_columns_to_convert + float_columns_to_convert].fillna(0)
    
    # Remove commas from the numbers and convert to appropriate type
    for col in integer_columns_to_convert:
        df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', ''), errors='coerce').fillna(0).astype('int64')
    
    for col in float_columns_to_convert:
        df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', ''), errors='coerce').fillna(0).astype('float64')
    
    return df

df = feature_engineering(df)
df.describe()

In [ ]:
table = PrettyTable()
table.field_names = ['Feature', 'Data Type']

for column in df.columns:
    column_dtype = str(df[column].dtype)
    table.add_row([column, column_dtype])

print(table)

# 3.) Data visualization

### 3.1) Top 30 most frequent and consistent Artists

*This horizontal bar plot visualizing the top 30 artists based on their frequency of appearance in the dataset provides valuable insights into artist productivity and prominence. By analyzing the number of times each artist appears, we can identify the most prolific or frequently featured artists in our dataset. This visualization contributes to our understanding of artist presence and productivity in the current music landscape, which can inform strategies for artist development, content curation, and platform management. It also provides a foundation for exploring how an artist's output volume relates to their overall success and impact in the industry.*

In [ ]:
## Top 20 artist based on how many times they appear

value_counts = df['Artist'].value_counts()
value_counts.head(30)
# Create the horizontal bar plot
plt.figure(figsize=(10, 6))
value_counts.head(30).plot(kind='barh')

# Add labels and title
plt.xlabel('Number of appearance')
plt.ylabel('Artist')
plt.title('Artist based on number of appearance')
plt.gca().invert_yaxis()  # Invert the y-axis to have the highest frequency on top
plt.grid(axis='x')
plt.show()


### 3.2) Top 20 Artists Based on Aggregated Performance Scores

*This visualization will help identify the most successful and popular artists in the dataset. By ranking the top 20 artists by their total Spotify streams, I can analyze which artists have the largest fan bases and the highest overall popularity. This information can be valuable for understanding the current landscape of the music industry, identifying potential industry leaders, and highlighting artists that may warrant further exploration or investment.*

In [ ]:
## Top 20 Artists based of sum of Spotify streams, Youtube views, Tiktok Views, Spotify Playlist Reach...........

artist_agg = df.groupby('Artist').agg({
    'Spotify Streams': 'sum',
    'YouTube Views': 'sum',
    'TikTok Views': 'sum',
    'Spotify Playlist Reach': 'sum',
    'YouTube Playlist Reach': 'sum',
    'Apple Music Playlist Count': 'sum',
    'Shazam Counts': 'sum'
})

artist_agg['Score'] = (
    artist_agg['Spotify Streams'] * 0.3 +
    artist_agg['YouTube Views'] * 0.3 +
    artist_agg['TikTok Views'] * 0.2 +
    artist_agg['Spotify Playlist Reach'] * 0.1 +
    artist_agg['YouTube Playlist Reach'] * 0.05 +
    artist_agg['Apple Music Playlist Count'] * 0.05 +
    artist_agg['Shazam Counts'] * 0.05
)

# Sort artists by the combined score and select the top 20
top_artists = artist_agg.sort_values(by='Score', ascending=False).head(20)

# Plot the top 20 artists
plt.figure(figsize=(12, 8))
top_artists['Score'].plot(kind='barh')
plt.xlabel('Score')
plt.ylabel('Artist')
plt.title('Top 20 Artists Based on Combined Score')
plt.gca().invert_yaxis()  # Highest score on top
plt.grid(axis='x')
plt.show()

### 3.3) Cross platform correlation visualization

*The cross-platform correlation heatmap will reveal the relationships between performance metrics across different music platforms. By visualizing these correlations, I can identify which platforms tend to have similar or dissimilar performance patterns. This insight can inform cross-platform marketing strategies, help predict success across platforms, and guide resource allocation for promotion efforts. Understanding these relationships will provide a more comprehensive view of how music performs in the current multi-platform digital landscape, enabling more effective decision-making for artists, labels, and platform managers.*

In [ ]:
# Extract relevant columns for correlation
correlation_data = df[['Spotify Streams', 'Spotify Playlist Count', 'Spotify Playlist Reach',
                       'YouTube Views', 'TikTok Views', 'YouTube Playlist Reach', 
                       'Apple Music Playlist Count', 'Deezer Playlist Reach', 
                       'Pandora Streams', 'Soundcloud Streams', 'Track Score', 
                       'Spotify Popularity']]

# Compute the correlation matrix
correlation_matrix = correlation_data.corr()

# Plot Heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=.5, vmin=-1, vmax=1)
plt.title('Correlation Between various platform counts')
plt.show()

### 3.4) Trends in Explicit Content Over Time

*This line chart illustrates the annual count of explicit tracks over several years, highlighting how the frequency of explicit content in music has changed over time. By tracking the number of explicit tracks each year, the graph reveals trends and shifts in the industry’s approach to explicit content, providing insight into how societal attitudes and industry standards around explicit material have evolved.*

In [ ]:
# Group by 'Year' and calculate the number of Explicit Tracks for each year
explicit_tracks = df[df['Explicit Track'] == 1]
explicit_tracks_per_year = explicit_tracks.groupby('Year')['Track'].count()

plt.figure(figsize=(15, 10))
explicit_tracks_per_year.plot(kind='line', marker='o')
plt.title('Number of Explicit Tracks per Year')
plt.xlabel('Year')
plt.ylabel('Number of Tracks')
plt.grid(True)
plt.show()

### 3.5) Change in Track Score Throughout the Years

*This line chart displays the average track score for each year, illustrating how the quality or popularity of tracks has evolved over time. By plotting the average track score annually, the chart provides insights into trends in music quality and listener reception, helping to identify whether there has been a general improvement or decline in track performance and how changing industry dynamics might be influencing these scores.*

In [ ]:
# Group by 'Year' and calculate the average 'Track Score' for each year
avg_track_score_per_year = df.groupby('Year')['Track Score'].mean()

plt.figure(figsize=(15, 10))
avg_track_score_per_year.plot(kind='line', marker='o')
plt.title('Change in Track Score Throughout the Years')
plt.xlabel('Year')
plt.ylabel('Average Track Score')
plt.grid(True)
plt.show()

### 3.6) Popularity Reach Across Top Platforms

*This 3D scatter plot visualizes the performance of tracks across three major platforms: Spotify, YouTube, and TikTok. The plot shows Spotify streams on the x-axis, YouTube views on the y-axis, and TikTok views on the z-axis, with the color of each point representing the track score. By examining this plot, you can identify patterns in how tracks perform across different platforms, reveal correlations between performance metrics on these platforms, and understand how high-scoring tracks fare in terms of reach and engagement across the major streaming services.*

In [ ]:
# Extract relevant columns
plot_data = df[['Spotify Streams', 'YouTube Views', 'TikTok Views', 'Track Score']].dropna()

# Prepare data for 3D plot
x = plot_data['Spotify Streams']
y = plot_data['YouTube Views']
z = plot_data['TikTok Views']
color = plot_data['Track Score']

# Create 3D scatter plot
fig = plt.figure(figsize=(14, 10))
ax = fig.add_subplot(111, projection='3d')

# Scatter plot
sc = ax.scatter(x, y, z, c=color, cmap='viridis', s=50, alpha=0.7, edgecolors='w')

# Add labels and title
ax.set_xlabel('Spotify Streams')
ax.set_ylabel('YouTube Views')
ax.set_zlabel('TikTok Views')
ax.set_title('3D Scatter Plot of Track Performance Across Platforms')

# Add color bar
cbar = plt.colorbar(sc, ax=ax, shrink=0.5, aspect=5)
cbar.set_label('Track Score')

plt.show()

### 3.7) Impact of Release Date on Track Popularity

*This line chart demonstrates how average track popularity, as measured by Spotify Popularity, varies by release year. By aggregating data to show the average popularity of tracks for each year, this visualization reveals trends in how the appeal of music tracks has evolved over time. The chart helps to identify whether there are increasing or decreasing trends in track popularity and how industry shifts, such as changes in musical trends or marketing strategies, may have influenced these patterns.*

In [ ]:
# Aggregate data by year to find average popularity
df_yearly = df.groupby('Year')['Spotify Popularity'].mean().reset_index()

plt.figure(figsize=(12, 6))
sns.lineplot(data=df_yearly, x='Year', y='Spotify Popularity', marker='o')
plt.title('Average Track Popularity by Release Year')
plt.xlabel('Release Year')
plt.ylabel('Average Spotify Popularity')
plt.grid(True)
plt.show()


### 3.8) Market Share of Music Discovery

*This pie chart illustrates the distribution of track counts across various music discovery platforms, including Spotify, YouTube, TikTok, Apple Music, Deezer, Amazon, Pandora, and Soundcloud. By visualizing the sum of track counts for each platform, the chart provides insights into how tracks are distributed across different services and highlights the relative market share of each platform in the music discovery landscape. This visualization helps to understand which platforms dominate in track discovery and engagement, guiding decisions on where to focus marketing efforts or where trends in music consumption are shifting.*

In [ ]:
# Sum of track counts per platform (example using Spotify Playlist Count, YouTube Playlist Reach, etc.)
platform_counts = {
    'Spotify': df['Spotify Playlist Count'].sum(),
    'YouTube': df['YouTube Views'].sum(),
    'TikTok': df['TikTok Posts'].sum(),
    'Apple Music': df['Apple Music Playlist Count'].sum(),
    'Deezer': df['Deezer Playlist Count'].sum(),
    'Amazon': df['Amazon Playlist Count'].sum(),
    'Pandora': df['Pandora Streams'].sum(),
    'Soundcloud': df['Soundcloud Streams'].sum()
}

plt.figure(figsize=(10, 10))

# Plot pie chart
plt.pie(platform_counts.values(), labels=platform_counts.keys(), autopct='%1.1f%%', colors=plt.cm.Paired(range(len(platform_counts))))
plt.title('Distribution of Tracks Across Platforms')
plt.show()

### 3.9) Artist Engagement Across Platforms

*This parallel coordinates plot visualizes how artists perform across multiple music platforms, including Spotify, YouTube, TikTok, and Apple Music. Each line represents an artist, with vertical axes representing performance metrics on each platform. By examining the plot, you can see how different artists compare in their engagement across these platforms, revealing patterns of consistent performance or highlighting those who excel in certain areas.*

In [ ]:
plot_data = df[['Artist', 'Spotify Streams', 'YouTube Views', 'TikTok Views', 'Apple Music Playlist Count']].dropna()

# Create a new DataFrame with average values for each artist
artist_engagement = plot_data.groupby('Artist').mean().reset_index()

# Plot
plt.figure(figsize=(14, 8))
parallel_coordinates(artist_engagement, 'Artist', colormap=plt.get_cmap('tab10'))
plt.title('Artist Engagement Across Platforms')
plt.xlabel('Platforms')
plt.ylabel('Average Values')
plt.legend(title='Artist', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.show()

### 3.10) Total Track Engagement Over Time

*This line plot shows the annual total engagement of tracks by aggregating Spotify streams, YouTube views, and TikTok views. The x-axis represents the year, and the y-axis shows the total combined engagement for all tracks in that year. This visualization helps identify trends and shifts in overall track engagement, providing insights into how audience interaction with tracks has changed over time.*

In [ ]:
# Aggregate data by year to find total engagement (e.g., combined streams and views)
df['Total Engagement'] = df['Spotify Streams'] + df['YouTube Views'] + df['TikTok Views']
df_yearly_engagement = df.groupby('Year')['Total Engagement'].sum().reset_index()

plt.figure(figsize=(12, 6))
sns.lineplot(data=df_yearly_engagement, x='Year', y='Total Engagement', marker='o')
plt.title('Total Track Engagement Over Time')
plt.xlabel('Year')
plt.ylabel('Total Engagement')
plt.grid(True)
plt.show()
